In [1]:
%load_ext autoreload
%autoreload 2

import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import ee
import geemap.foliumap as geemap
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model

In [2]:
ee.Authenticate()
ee.Initialize(project="215656163750")
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
import utils
import explore

Mounted at /content/drive


In [3]:
# The fine-tuned model from initialModel.ipynb

model_type = "VGG16"
model = load_model('/content/drive/MyDrive/CAFO_models/initialVGG16_step2.keras')

In [4]:
# Functions for data preparation

data_path = "/content/drive/MyDrive/CAFO_data/forTraining/"

def read_data(dataset):

  with open(f'{data_path}{dataset}_sentinel_images.pkl', 'rb') as f:
      image_dict = pickle.load(f)

  df = pd.read_pickle(f'{data_path}{dataset}_metadata_gdf.pkl')
  df["Sentinel"] = image_dict.values()

  return df


def prepare(df):

  images = []
  labels = []

  for idx, row in df.iterrows():
    if row['Farm type'] in ["Poultry", "Broiler", "Layer", "Pig", "Unknown CAFO"]:
      labels.append("CAFO")
      images.append(row["Sentinel"])
    elif row['Farm type'] == "Non-farm":
      labels.append("Non-farm")
      images.append(row["Sentinel"])
    else:
      print(f"Unexpected farm type, {row['Farm type']}")

  label_encoder = LabelEncoder()
  encoded_labels = label_encoder.fit_transform(labels)
  one_hot_labels = to_categorical(encoded_labels, num_classes=len(set(labels)))

  class_mapping = dict(zip(encoded_labels, labels))
  print("Class mapping:", class_mapping)

  return images, one_hot_labels

In [5]:
# Main function for finding images that need checking

def find_dodgy_images(dataset, model_type, label_type="false negative", min_prob=0.7):

  # Get the Sentinel images etc. for this country/location
  data = read_data(dataset)
  X, y = prepare(data)
  meta = data.drop(columns=["Sentinel"])

  # Apply the model to all images
  results = utils.get_predictions(model, X, y, model_type, meta)

  if label_type == "false negative":
    # Create a new column conveniently displaying CAFO probability
    results.loc[:, "CAFO prob"] = results.loc[:, "Model Probabilities"].str[0]
    # Identify images labeled as not-farm but predicted to be CAFOs
    to_check = results[(results["Label"] == 1) & (results["CAFO prob"] >= min_prob)]
    # Sort in descending probability order
    to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)
  elif label_type == "false positive":
    # Create a new column conveniently displaying not-farm probability
    results.loc[:, "not-farm prob"] = results.loc[:, "Model Probabilities"].str[1]
    # Identify images labeled as CAFO but predicted to be not-farms
    to_check = results[(results["Label"] == 0) & (results["not-farm prob"] >= min_prob)]
    # Sort in descending probability order
    to_check.sort_values(by="not-farm prob", ascending=False, inplace=True)
  else:
    raise ValueError("label_type must be either 'false negative' or 'false positive'")

  print(f"There are {len(to_check)} images to check")
  print("here")
  return data, to_check

## Mexico

Start by examining images labeled not-CAFO but predicted to be CAFOs.

In [ ]:
mex, check_mex = find_dodgy_images("mex", model_type, 0.5)

Class mapping: {0: 'CAFO', 1: 'Non-farm'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


210/210 ━━━━━━━━━━━━━━━━━━━━ 419s 2s/step
There are 444 images to check


<ipython-input-5-4ab290b480b9>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)


In [ ]:
# Examine those images, flag ones that really are CAFOs

#mislabeled_1 = explore.loop_over_buildings(check_mex[:50])
#...
mislabeled_9 = explore.loop_over_buildings(check_mex[400:])

Working on feature 1 of 44


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,CAFO prob
3801,1,"[0.537051, 0.46294898]",0,México,Guanajuato,Ocampo,1499.14624,1.31805,Mexico,Non-farm,45.069657,NaN,POINT (-101.46134 21.78437),0.537051


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 2 of 44


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,CAFO prob
3412,1,"[0.53683263, 0.4631673]",0,México,Jalisco,Mazamitla,1111.361572,2.435843,Mexico,Non-farm,51.782539,NaN,POINT (-103.08442 19.86144),0.536833


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 3 of 44


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 43 of 44


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,CAFO prob
2404,1,"[0.501342, 0.49865797]",0,México,Sinaloa,Guasave,1039.272461,8.254908,Mexico,Non-farm,92.805317,None,POINT (-108.78198 25.72636),0.501342


Enter reject to reject, exit to exit, or any key to continue  reject
Working on feature 44 of 44


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,CAFO prob
335,1,"[0.5007268, 0.49927324]",0,México,México,Amecameca,2371.1215,1.522753,Mexico,Non-farm,65.012732,None,POINT (-98.7496 19.15024),0.500727


Enter reject to reject, exit to exit, or any key to continue  
7


In [ ]:
mex_mislabeled = mislabeled_1 + mislabeled_2 + mislabeled_3 + mislabeled_4 +\
 mislabeled_5 + mislabeled_6 + mislabeled_7 + mislabeled_8 + mislabeled_9

print(f"Rejected {len(mex_mislabeled)} / {len(check_mex)} images")

Rejected 163 / 444 images


Now examine Mexico images labeled CAFO but predicted to be not-farm

In [6]:
_, check_mex2 = find_dodgy_images("mex", model_type,\
                                  label_type="false positive",\
                                  min_prob=0.5)

Class mapping: {0: 'CAFO', 1: 'Non-farm'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


210/210 ━━━━━━━━━━━━━━━━━━━━ 430s 2s/step
There are 442 images to check
here


<ipython-input-5-f8ba9b54087e>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="not-farm prob", ascending=False, inplace=True)


In [25]:
additional_1 = explore.loop_over_buildings(check_mex2[:50], column="Parent coords")
#...
#additional_9 = explore.loop_over_buildings(check_mex2[400:], column="Parent coords")

Working on feature 1 of 42


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,not-farm prob
4883,0,"[0.4743289, 0.5256711]",1,México,Jalisco,San Pedro Tlaquepaque,5110.671387,1.003962,Mexico,Pig,76.290453,"POLYGON ((-103.40528 20.58839, -103.40528 20.5...",0.525671


Enter reject to reject, exit to exit, or any key to continue  reject
Working on feature 2 of 42


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,not-farm prob
5754,0,"[0.47616854, 0.5238314]",1,México,Jalisco,Tlajomulco de Zúñiga,20177.3234,1.398439,Mexico,Pig,192.026362,"POLYGON ((-103.47432 20.44458, -103.47432 20.4...",0.523831


Enter reject to reject, exit to exit, or any key to continue  reject
Working on feature 3 of 42


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 41 of 42


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,not-farm prob
1307,0,"[0.4997772, 0.5002228]",1,México,Sonora,Cajeme,2642.1454,1.580302,Mexico,Pig,64.917844,"POLYGON ((-109.92952 27.41053, -109.92952 27.4...",0.500223


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 42 of 42


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,not-farm prob
5320,0,"[0.4999854, 0.50001466]",1,México,México,Tenancingo,12956.4427,1.207925,Mexico,Pig,151.781616,"POLYGON ((-99.59492 18.94655, -99.59493 18.946...",0.500015


Enter reject to reject, exit to exit, or any key to continue  


In [28]:
additional_1 = [6524, 4560, 2630, 1662, 5426, 2350, 6611, 131, 5777, 6670, 132, 133, 134, 4886, 6543, 6283, 5143, 2103, 163, 6282, 1661, 1877, 1432, 1753, 40, 872, 5449, 6525, 4999, 1754, 3091, 3574, 1188, 5600, 1413, 1596, 41, 392, 2349, 3099, 5753]
additional_2 = [14, 3635, 1435, 4562, 4036, 5554, 2376, 1200, 1519, 6509, 2518, 3090, 1650, 5716, 24, 3299, 6526, 6334, 6502, 6669, 562, 4375, 5949, 4884, 784, 324, 1540, 6593, 727, 3645, 1485, 3162, 1424, 1614]
additional_3 = [364, 6344, 5647, 4872, 5352, 4034, 6294, 363, 488, 2487, 559, 5630, 365, 4987, 1170, 135, 920, 5991, 6661, 776, 6619, 2316, 1989, 1993, 1992, 1991, 1990, 1988, 1054, 6144, 563, 4193, 5952, 362, 753, 366, 953]
additional_4 = [5900, 774, 3472, 6657, 6656, 6655, 2558, 2847, 589, 2375, 1484, 5111, 1022, 6599, 6366, 4053, 1527, 4656, 6481, 6320, 569, 4102, 5010, 458, 6658, 976, 11, 4430, 1154]
additional_5 = [4147, 5948, 4663, 109, 5265, 4660, 3276, 5713, 6508, 2069, 5592, 4823, 4199, 729, 2955, 6671, 2547, 6416, 5177, 4635, 5356, 3268, 671, 9, 1523, 5180, 5175, 5756]
additional_6 = [2877, 2878, 5692, 6544, 4954, 931, 1723, 6517, 6204, 5556, 1173, 1174, 4039, 3282, 4086, 2849, 2374, 5562, 5011, 998, 3723]
additional_7 = [2631, 4408, 6174, 5767, 6634, 6170, 6500, 1737, 4712, 4443, 212, 3050, 6437, 4711, 919, 1362, 1745, 2879, 1187, 1186, 4694]
additional_8 = [4871, 6534, 2983, 2982, 2648, 2751, 2033, 2023, 3624, 6621, 1001, 4709, 4710, 4553, 4537, 1093, 5148, 5009, 5361, 996, 4044]
additional_9 = [4883, 5754, 2046, 2984, 5917, 2348, 4726, 1014, 2339, 3654, 2366, 5769, 2907, 775, 608, 1010]

mex_additional = additional_1 + additional_2 + additional_3 + additional_4 +\
                 additional_5 + additional_6 + additional_7 + additional_8 + additional_9

print(f"Rejected {len(mex_additional)} / {len(check_mex2)} images")

Rejected 248 / 442 images


Change the labels of the mislabeled CAFO images in the original df, drop the farms that weren't really farms,and pickle the cleaned df with a new name.

In [35]:
df = mex.copy()
df.loc[df.index.isin(mex_mislabeled), ['Farm type']] = 'Unknown CAFO'
df.drop(mex_additional, axis='index', inplace=True)
df.to_pickle(f'{data_path}mex_final.pkl')

,ADM0,ADM1,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Parent coords,geometry,Sentinel
0,México,Yucatán,Abalá,1278.260864,6.201277,Mexico,Pig,88.561704,"POLYGON ((-89.618592 20.686914, -89.618611 20....",POINT (-89.62289 20.68564),"[[[70.94526368085846, 58.41501567171387, 43.15..."
1,México,Yucatán,Abalá,1251.505371,6.277562,Mexico,Pig,88.270053,"POLYGON ((-89.670089 20.695356, -89.670109 20....",POINT (-89.67311 20.69408),"[[[57.234060587310644, 49.81131783246348, 35.2..."
2,México,Jalisco,Acatlán de Juárez,1910.770200,8.406895,Mexico,Pig,126.224795,"POLYGON ((-103.600494 20.378479, -103.600499 2...",POINT (-103.60499 20.37752),"[[[77.28670712134621, 62.57250170754432, 43.47..."
3,México,Jalisco,Acatlán de Juárez,1910.770200,8.406895,Mexico,Broiler,126.224795,"POLYGON ((-103.600498 20.379905, -103.600503 2...",POINT (-103.60499 20.37752),"[[[77.28670712134621, 62.57250170754432, 43.47..."
4,México,Jalisco,Acatlán de Juárez,1576.872070,7.293715,Mexico,Broiler,126.057422,"POLYGON ((-103.588165 20.412613, -103.588171 2...",POINT (-103.59188 20.41013),"[[[170.9339196568565, 130.7831706186417, 101.9..."
5,México,Jalisco,Acatlán de Juárez,2140.873900,10.738451,Mexico,Broiler,152.057494,"POLYGON ((-103.584105 20.417175, -103.584111 2...",POINT (-103.58694 20.41552),"[[[90.71809472250027, 64.5485667482669, 46.379..."
6,México,Jalisco,Acatlán de Juárez,1891.812134,9.298157,Mexico,Unknown CAFO,133.362645,None,POINT (-103.57391 20.41651),"[[[70.67224344695425, 61.866041667413675, 41.9..."
7,México,Guanajuato,Acámbaro,9255.619700,1.258559,Mexico,Pig,114.808831,"POLYGON ((-100.665581 20.029507, -100.665589 2...",POINT (-100.66984 20.02798),"[[[43.842236116709714, 38.05074250835954, 25.0..."
8,México,Guanajuato,Acámbaro,1621.561890,6.831521,Mexico,Non-farm,105.223567,None,POINT (-100.72966 20.03993),"[[[47.03448060855627, 36.82202128868557, 28.53..."
10,México,Guanajuato,Acámbaro,2295.463500,1.169443,Mexico,Non-farm,51.923547,None,POINT (-100.78401 20.06598),"[[[70.6312521541026, 55.00683149418119, 42.645..."


## Chile

In [ ]:
chl, check_chl = find_dodgy_images("chl", model_type, 0.5)

Class mapping: {1: 'Non-farm', 0: 'CAFO'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


21/21 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step
There are 51 images to check


<ipython-input-5-4ab290b480b9>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)


In [ ]:
chl_mislabeled = explore.loop_over_buildings(check_chl)

Working on feature 1 of 51


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,ADM3,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Number of animals,Parent coords,CAFO prob
309,1,"[0.92240006, 0.07759991]",0,Chile,Región del Maule,Curicó,Hualañé,1742.395752,6.871482,Chile,Non-farm,112.18273,POINT (-71.76753 -34.99095),NaN,None,0.9224


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 2 of 51


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,ADM3,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Number of animals,Parent coords,CAFO prob
588,1,"[0.9208433, 0.07915672]",0,Chile,Región de Ñuble,Diguillín,Chillán Viejo,1618.207031,1.274267,Chile,Non-farm,46.230223,POINT (-72.22073 -36.6355),NaN,None,0.920843


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 3 of 51


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 50 of 51


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,ADM3,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Number of animals,Parent coords,CAFO prob
415,1,"[0.51000875, 0.4899912]",0,Chile,Región Metropolitana de Santiago,Maipo,San Bernardo,1230.481201,1.044236,Chile,Non-farm,38.011291,POINT (-70.72593 -33.57323),NaN,None,0.510009


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 51 of 51


,Label,Model Probabilities,Model Class,ADM0,ADM1,ADM2,ADM3,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Number of animals,Parent coords,CAFO prob
409,1,"[0.50611883, 0.4938812]",0,Chile,Región Metropolitana de Santiago,Maipo,San Bernardo,868.248413,5.633691,Chile,Non-farm,71.981826,POINT (-70.72459 -33.60898),NaN,None,0.506119


Enter reject to reject, exit to exit, or any key to continue  reject


In [ ]:
print(f"Rejected {len(chl_mislabeled)} / {len(check_chl)} images")

df = chl.copy()
df.loc[df.index.isin(chl_mislabeled), ['Farm type']] = 'Unknown CAFO'

df.to_pickle(f'{data_path}chl_final.pkl')

Rejected 2 / 51 images


## Iowa

In [ ]:
iowa, check_iowa = find_dodgy_images("iowa", model_type, 0.5)

Class mapping: {1: 'Non-farm', 0: 'CAFO'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


205/205 ━━━━━━━━━━━━━━━━━━━━ 410s 2s/step
There are 190 images to check


<ipython-input-5-4ab290b480b9>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)


In [ ]:
iowa_mislabeled = explore.loop_over_buildings(check_iowa)

Working on feature 1 of 190


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Number of animals,Parent coords,geometry,CAFO prob
2058,1,"[0.9972627, 0.002737296]",0,Palo Alto,1473.840297,5.186529,Iowa,Non-farm,87.524514,NaN,None,POINT (-94.91388 42.92137),0.997263


Enter reject to reject, exit to exit, or any key to continue  reject
Working on feature 2 of 190


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Number of animals,Parent coords,geometry,CAFO prob
827,1,"[0.9923603, 0.0076397737]",0,Story,1835.515791,9.474145,Iowa,Non-farm,131.983486,NaN,None,POINT (-93.3615 41.86315),0.99236


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 3 of 190


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 60 of 190


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Number of animals,Parent coords,geometry,CAFO prob
1427,1,"[0.7938949, 0.20610505]",0,Fayette,1061.118934,2.160462,Iowa,Non-farm,55.491949,NaN,None,POINT (-92.04341 42.89891),0.793895


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 61 of 190


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),Number of animals,Parent coords,geometry,CAFO prob
4806,1,"[0.7913318, 0.20866813]",0,Lyon,1850.537122,5.483907,Iowa,Non-farm,101.044665,NaN,None,POINT (-96.44281 43.30046),0.791332


Enter reject to reject, exit to exit, or any key to continue  exit


In [ ]:
print(f"Rejected {len(iowa_mislabeled)} / {len(check_iowa)} images")

df = iowa.copy()
df.loc[df.index.isin(iowa_mislabeled), ['Farm type']] = 'Unknown CAFO'

df.to_pickle(f'{data_path}iowa_final.pkl')

Rejected 14 / 190 images


## Romania


In [ ]:
rou, check_rou = find_dodgy_images("rou", model_type, 0.5)

Class mapping: {1: 'Non-farm', 0: 'CAFO'}


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


26/26 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step
There are 43 images to check


<ipython-input-5-4ab290b480b9>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_check.sort_values(by="CAFO prob", ascending=False, inplace=True)


In [ ]:
rou_mislabeled = explore.loop_over_buildings(check_rou)

Working on feature 1 of 43


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Parent coords,CAFO prob
689,1,"[0.9803991, 0.019600928]",0,IZVORU BARZII,2181.708451,6.311394,Romania,Non-farm,118.130982,POINT (22.69409 44.66024),None,0.980399


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 2 of 43


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Parent coords,CAFO prob
281,1,"[0.96017665, 0.039823398]",0,BOD,3227.379087,6.407257,Romania,Non-farm,146.81952,POINT (25.59889 45.75942),None,0.960177


Enter reject to reject, exit to exit, or any key to continue  reject
Working on feature 3 of 43


...

Enter reject to reject, exit to exit, or any key to continue  
Working on feature 20 of 43


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Parent coords,CAFO prob
583,1,"[0.7415495, 0.25845048]",0,MEDIESU AURIT,2003.82855,2.001201,Romania,Non-farm,84.618496,POINT (23.14424 47.7791),None,0.741549


Enter reject to reject, exit to exit, or any key to continue  
Working on feature 21 of 43


,Label,Model Probabilities,Model Class,ADM2,Area (sq m),Aspect ratio,Dataset name,Farm type,Length (m),geometry,Parent coords,CAFO prob
500,1,"[0.73040646, 0.2695935]",0,SUHARAU,1890.301507,1.869325,Romania,Non-farm,88.647714,POINT (26.47906 48.09827),NaN,0.730406


Enter reject to reject, exit to exit, or any key to continue  exit


In [ ]:
# Relabel the probably-CAFOs and write file

print(f"Rejected {len(rou_mislabeled)} / {len(check_rou)} images")

df = rou.copy()
df.loc[df.index.isin(rou_mislabeled), ['Farm type']] = 'Unknown CAFO'

df.to_pickle(f'{data_path}rou_final.pkl')

Rejected 6 / 43 images
